### Journey Planner 

hypotheses:

    i) given the arcs A->B, if ta_dep < tb_dep => ta_arr < tb_arr (before you depart before you arrive)
    ii) delays distribute with lognorm


simplifications:
    - the number of changes is not taken into account (one would prefer also to take do few stops)


In [2]:
from scipy.stats import lognorm
import pandas as pd
import numpy as np

from pyscripts.test_utils import rand_edges
from pyscripts.planner import *

%load_ext autoreload
%autoreload 2

In [61]:
class Planner():
    prob_one = 0.99 # approximation of probability 1, used to prune the choices
    
    def __init__(self, edges, reverse=False):
        """ 
        Input:
        - edges: DataFrame indicating all the edges (edges) between nodes.
        - reverse: boolean indicating wether the graph has to be explored in a direct of reversed mode, i.e. if the exploration starts from the departure node (False) or from the arrival node (True).
        """
        
        self.edges = edges
        self.reverse = reverse
    
    def clear(self):
        """ Clear the structure so to restart the computation of the best path. """
        self.edges.prob = np.NaN
        self.edges.label = Status.Unvisited
    
    def compute_plan(self, departure_node, arrival_node, time, treshold):
        """ Given the names of the departure and arrival nodes, a treshold and the time (which may refer either to the departure or arrival time depending on self.reverse), computes the edges above the treshold. """
        
        start_node = arrival_node if self.reverse else departure_node
        # list of Visited edges
        visited_ids = []

        # ------------ Initialization ------------ 
        # visit the edges from the starting node
        visited_ids = self.initialize_from(start_node, time)

        # ------------ Main iteration -------------
        while True:
            # get node to be expanded
            edge_id = self.best_node(visited_ids)

            # check termination: we reached the target node and with a edge which is the best one
            if self.done(edge_id, target_node):
                break

            # expand it
            visited_ids = visited_ids[visited_ids != edge_id] # remove the id we are going to expand
            newly_visited = self.expand_edge(edge_id, treshold=treshold) # expand
            visited_ids = np.append(visited_ids, newly_visited) # update the list of visited nodes
        
        last_edge_id = edge_id
        
        # ------------ Collect the edges id of the found path and return them -------------
        node_col = "arr_name" if self.reverse else "dep_name"
        edge = self.edges.loc[last_edge_id]

        path = [last_edge_id]
        while not pd.isnull(edge.prev_edge):
            edge = self.edges.loc[edge.prev_edge]
            path.append(edge.name)
            
        if not self.reverse:
            path.reverse()
            
        return path
        
    def initialize_from(self, node_name, time):
        """ 
        Initializes the search given a starting node and time.
        Fetches for all edges <node_name> -> A (where A is a reachable node) the one that starts the first and and set it as Visited with probability 1. 
        Note: the meaning of 'starts first' has different meaning depending on the value of self.reverse: 
            - self.reverse = False: search for the first departing edges after the 'time'
            - self.reverse = True : search for the first arriving edges before the 'time'
        return the ids of the visited edges.
        """
        # here we are applying the aforementioned hypothesis i)
        
        # select the ids of the edges
        if self.reverse:
            time_col = "arr_time"
            name_col = "arr_name"
            
            ids = (self.edges[(self.edges[name_col] == node_name) & (self.edges[time_col] < time)]
                .groupby(["dep_name", "arr_name"])[time_col]
                .idxmax().values)
        else:
            time_col = "dep_time"
            name_col = "dep_name"
            ids = (self.edges[(self.edges[name_col] == node_name) & (self.edges[time_col] > time)]
                .groupby(["dep_name", "arr_name"])[time_col]
                .idxmin().values)
                    
        # visit the selected edges with probability 1
        self.edges.loc[ids, ["prob", "label"]] = [1, Status.Visited]
        return ids
        
    def expand_edge(self, edge_id, treshold=0.8):
        """ 
        Given a edge id, expand all the edges starting from the reached node whose probability is higher than the given treshold. 
        To prune we consider only the edges from B to A in ascending order of time and stop when there is at least a edge whose probability of taking it is >= Planner.prob_one.
        Note: the meaning of 'reach' has different meaning depending on the value of self.reverse: 
            - self.reverse = False: scan the edges departing after the time and from the station specified by the edge_id 
            - self.reverse = True : scan the edges arriving before the time and to the station specified by the edge_id 
        """
        
        # set the edge as expanded 
        self.edges.loc[edge_id, "label"] = Status.Expanded
        curr_edge = self.edges.loc[edge_id] 
        
        if self.reverse:
            # define direction of the edges
            from_time_col = "arr_time"
            to_time_col = "dep_time"
            
            from_node_col = "arr_name"
            to_node_col = "dep_name"
            
            # define the constraint between the time in the 'from_time_col' column and the current time
            time_constraint = lambda arr_time, curr_time: arr_time < curr_time
            # specify how to order the edges when scanning them to compute the probabilities
            ascending = False
            
            # given a next possible edge compute the probability of taking it
            def compute_prob(next_edge):
                max_delay = curr_time - next_edge.arr_time
                prob = lognorm(next_edge.std_delay, next_edge.mean_delay).cdf(max_delay)
                prob *= curr_edge.prob
                return prob
        else:            
            # define direction of the edges
            from_time_col = "dep_time"
            to_time_col = "arr_time"
            
            from_node_col = "dep_name"
            to_node_col = "arr_name"
            
            # define the constraint between the time in the 'from_time_col' column and the current time
            time_constraint = lambda dep_time, curr_time: dep_time > curr_time
            # specify how to order the edges when scanning them to compute the probabilities
            ascending = True
            
            # given a next possible edge compute the probability of taking it
            def compute_prob(next_edge):
                max_delay = next_edge.dep_time - curr_time
                prob = lognorm(curr_edge.std_delay, curr_edge.mean_delay).cdf(max_delay)
                prob *= curr_edge.prob
                return prob
            
        # get current node and time
        curr_node = curr_edge[to_node_col]
        curr_time = curr_edge[to_time_col]
            
        # TODO: check day (select only current and next days)
        # select all the possible candidate edges
        # get also the aready visited edges, we will try to improve their probability
        candidates = self.edges[
            (self.edges[from_node_col] == curr_node) & 
            (time_constraint(self.edges[from_time_col], curr_time))
        ]

        # for each next possible node scan all the possible edges until the probability of taking the edge is above Planner.prob_one
        nodes = candidates[to_node_col].unique() # get the set of possible next nodes
        ids = []
        for next_node in nodes:
            possible_edges = candidates[candidates[to_node_col] == next_node].sort_values(from_time_col, ascending=ascending)
            for idx, edge in possible_edges.iterrows():
                # TODO: check if the column trip_id remains the same (if so just propagate the prob)
                prob = compute_prob(edge)

                if prob > treshold:
                    # if above the treshold then I could take (visit) this edge
                    
                    # if it is the first time you visit the edge (label==Unvisited) then just visit this edge
                    # otherwise we also check if we can improve the probability (if so then we re-take this edge)
                    if edge.label == Status.Unvisited or (edge.label != Status.Unvisited and prob > edge.prob):
                        ids.append(idx)
                        self.edges.loc[idx, "label"] = Status.Visited
                        self.edges.loc[idx, "prob"] = prob
                            
                if prob > Planner.prob_one:
                    # we found at least one edge to the node whose probability is high enough
                    break
        
        self.edges.loc[ids, "prev_edge"] = curr_edge.name # set the expanded edge as previous to the newly visited ones
        return ids
    
    def best_node(self, ids):
        """ Given a list of ids corresponding to al the Visited edges (for efficiency) returns the id of the best edge (the one we have to expand)."""        
        # the best node (the one to be expanded) is:
        if self.reverse:
            # the one with the highest departure time
            return int(self.edges.loc[ids, "dep_time"].idxmax())
        else:
            # the one with the lowest arrival time
            return int(self.edges.loc[ids, "arr_time"].idxmin())
    
    def done(self, best_edge_id, target_node):
        """ Given the id of the best edge and the name of the target node check if we reached it. """
        if self.reverse:
            return self.edges.loc[best_edge_id, "dep_name"] == target_node
        else:
#             print(self.edges.loc[best_edge_id, "arr_name"], target_node)
            return self.edges.loc[best_edge_id, "arr_name"] == target_node

In [65]:
# planner = Planner(rand_edges(0.3, 0.2), reverse=False)
planner = Planner(rand_edges(0.1, 0.2), reverse=False)
# planner = Planner(rand_edges(), reverse=False)

start_node = "name1"
target_node = "name5"
start_time = 0.0
treshold = 0.85

last_id = planner.compute_plan(start_node, target_node, start_time, treshold)

print("Path:", last_id)
planner.edges

Path: [6, 9, 5]


,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Unvisited,0.2,0.1,NaN
1,name1,name2,0.1,1.5,1,1.000000,Expanded,0.2,0.1,NaN
2,name2,name4,3.0,4.0,4,0.995650,Expanded,0.2,0.1,1.0
3,name2,name4,2.0,3.0,3,NaN,Unvisited,0.2,0.1,NaN
4,name4,name5,5.0,7.0,6,NaN,Unvisited,0.2,0.1,NaN
5,name4,name5,4.0,6.0,5,0.991319,Visited,0.2,0.1,9.0
6,name1,name3,0.5,1.0,7,1.000000,Expanded,0.2,0.1,NaN
7,name3,name5,5.0,8.0,8,1.000000,Visited,0.2,0.1,6.0
8,name3,name5,6.0,9.0,9,NaN,Unvisited,0.2,0.1,NaN
9,name3,name4,2.5,2.5,10,0.995650,Expanded,0.2,0.1,6.0


## Test Planner

- `initialize_from`

In [7]:
# direct search
planner = Planner(rand_edges(), reverse=False)

sol = [1, 6]
stop_name = "name1"
time = 0.0

ids = planner.initialize_from(stop_name, time)
display(planner.edges.loc[sol])
assert set(sol) == set(ids)

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
1,name1,name2,0.1,1.5,1,1.0,Visited,-1,1.000000e-10,NaN
6,name1,name3,0.5,1.0,7,1.0,Visited,-1,1.000000e-10,NaN


In [8]:
# reverse search
planner = Planner(rand_edges(), reverse=True)

sol = [8, 4]
planner.clear()
stop_name = "name5"
time = 10

ids = planner.initialize_from(stop_name, time)
display(planner.edges.loc[sol])
assert set(sol) == set(ids)

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
8,name3,name5,6.0,9.0,9,1.0,Visited,-1,1.000000e-10,NaN
4,name4,name5,5.0,7.0,6,1.0,Visited,-1,1.000000e-10,NaN


- `expand_edge`

In [9]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)
planner.edges.dep_name = "name2" # just to increase the possible choices

planner.edges.loc[1, "prob"] = 1
ids = planner.expand_edge(1) # expand the edge with id 1

sol = [2, 5]
assert set(sol) == set(ids)

planner.edges.sort_values("arr_name")

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name2,name2,0.5,2.0,-2,NaN,Unvisited,0.2,0.3,NaN
1,name2,name2,0.1,1.5,1,1.000000,Expanded,0.2,0.3,NaN
6,name2,name3,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,0.809090,Visited,0.2,0.3,1.0
3,name2,name4,2.0,3.0,3,NaN,Unvisited,0.2,0.3,NaN
9,name2,name4,2.5,2.5,10,NaN,Unvisited,0.2,0.3,NaN
4,name2,name5,5.0,7.0,6,NaN,Unvisited,0.2,0.3,NaN
5,name2,name5,4.0,6.0,5,0.997251,Visited,0.2,0.3,1.0
7,name2,name5,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name2,name5,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN


In [12]:
# reverse search
planner = Planner(rand_edges(0.3, 0.2), reverse=True)
planner.edges.arr_name = "name4" # just to increase the possible choices

planner.edges.loc[4, "prob"] = 1
ids = planner.expand_edge(4) # expand the edge with id 4

sol = [0, 3, 9]
assert set(sol) == set(ids)

planner.edges.sort_values("dep_name")

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name4,0.5,2.0,-2,0.999700,Visited,0.2,0.3,4.0
1,name1,name4,0.1,1.5,1,NaN,Unvisited,0.2,0.3,NaN
6,name1,name4,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,NaN,Unvisited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,0.974961,Visited,0.2,0.3,4.0
7,name3,name4,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name3,name4,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN
9,name3,name4,2.5,2.5,10,0.997251,Visited,0.2,0.3,4.0
4,name4,name4,5.0,7.0,6,1.000000,Expanded,0.2,0.3,NaN
5,name4,name4,4.0,6.0,5,NaN,Unvisited,0.2,0.3,NaN


- `best_node`

In [18]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)
visited = np.array([0, 3, 5, 8])

planner.edges.loc[visited, "label"] = Status.Visited

best = planner.best_node(visited)
sol = 0
assert sol == best

planner.edges[planner.edges.label == Status.Visited]

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Visited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Visited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Visited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Visited,0.2,0.3,NaN


In [19]:
# reverse search
planner = Planner(rand_edges(0.3, 0.2), reverse=True)
visited = np.array([0, 3, 5, 8])

planner.edges.loc[visited, "label"] = Status.Visited

best = planner.best_node(visited)
sol = 8
assert sol == best

planner.edges[planner.edges.label == Status.Visited]

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Visited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Visited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Visited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Visited,0.2,0.3,NaN


- `done`

In [31]:
# direct search
planner = Planner(rand_edges(0.3, 0.2), reverse=False)

assert planner.done(2, "name4")
assert not planner.done(2, "name1")
assert planner.done(6, "name3")
assert not planner.done(8, "name1")

# reverse search
planner.reverse = True
assert planner.done(2, "name2")
assert not planner.done(2, "name1")
assert planner.done(6, "name1")
assert not planner.done(8, "name4")

planner.edges

,dep_name,arr_name,dep_time,arr_time,trip_id,prob,label,mean_delay,std_delay,prev_edge
0,name1,name2,0.5,2.0,-2,NaN,Unvisited,0.2,0.3,NaN
1,name1,name2,0.1,1.5,1,NaN,Unvisited,0.2,0.3,NaN
2,name2,name4,3.0,4.0,4,NaN,Unvisited,0.2,0.3,NaN
3,name2,name4,2.0,3.0,3,NaN,Unvisited,0.2,0.3,NaN
4,name4,name5,5.0,7.0,6,NaN,Unvisited,0.2,0.3,NaN
5,name4,name5,4.0,6.0,5,NaN,Unvisited,0.2,0.3,NaN
6,name1,name3,0.5,1.0,7,NaN,Unvisited,0.2,0.3,NaN
7,name3,name5,5.0,8.0,8,NaN,Unvisited,0.2,0.3,NaN
8,name3,name5,6.0,9.0,9,NaN,Unvisited,0.2,0.3,NaN
9,name3,name4,2.5,2.5,10,NaN,Unvisited,0.2,0.3,NaN


- `compute_plan`

In [ ]:
# # direct search
# trials = [
#     (Planner(rand_edges(0.3, 0.2), reverse=False), 5),
#     (Planner(rand_edges(0.1, 0.2), reverse=False), 4),
#     (Planner(rand_edges(), reverse=False), 5)    
# ]


# start_node = "name1"
# target_node = "name5"
# start_time = 0.0
# treshold = 0.85

# for trial in trials:
#     planner = trial[0]
#     sol = trial[1]
#     last_id = planner.compute_plan(start_node, target_node, start_time, treshold)
#     break
#     print(sol)
# planner.edges

In [ ]:
# start_node = "node1"

# node_col = "dep_name"
# edge = planner.edges.loc[sol]

# path = [sol]
# while not pd.isnull(edge.prev_edge):
#     edge = planner.edges.loc[edge.prev_edge]
#     path.append(edge.name)
# path.reverse()
# path